####  1. DATA 불러오기

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./data/bread_data.csv',encoding = 'cp949', low_memory=False)

In [3]:
df.dtypes.to_frame().reset_index() #일련번호부여

,index,0
0,번호,int64
1,개방서비스명,object
2,개방서비스id,object
3,개방자치단체코드,int64
4,관리번호,object
5,인허가일자,int64
6,인허가취소일자,float64
7,영업상태구분코드,int64
8,영업상태명,object
9,상세영업상태코드,int64


In [4]:
df.shape

(52680, 48)

In [5]:
df.head(1)

,번호,개방서비스명,개방서비스id,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,...,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지,Unnamed: 47
0,1,제과점영업,07_22_18_P,3600000,3600000-121-2013-00002,20130201,NaN,1,영업/정상,1,...,NaN,NaN,NaN,NaN,N,146.4,NaN,NaN,NaN,NaN


#### 2. 전처리

In [6]:
# 상세영업상태코드, 상세영업상태명 개수 추출
df['상세영업상태코드'].value_counts() #normalize=True쓰면 비율이 나옴  

2    33669
1    19011
Name: 상세영업상태코드, dtype: int64

In [7]:
#1 : 영업, #2 : 폐업
df['상세영업상태명'].value_counts()

폐업    33669
영업    19011
Name: 상세영업상태명, dtype: int64

In [8]:
df['폐업일자'].value_counts(normalize=True,dropna=False)

NaN           0.360877
20030402.0    0.000911
20201228.0    0.000892
20051229.0    0.000816
20060216.0    0.000797
                ...   
20000612.0    0.000019
20031104.0    0.000019
20010823.0    0.000019
20040108.0    0.000019
20210207.0    0.000019
Name: 폐업일자, Length: 5689, dtype: float64

In [9]:
# 컬럼 하나하나 살펴봐야 하지만 생략하고 필요 컬럼만 추출
bread = df[['인허가일자','상세영업상태코드','폐업일자','소재지전체주소','사업장명','업태구분명']]
bread.head(3)

,인허가일자,상세영업상태코드,폐업일자,소재지전체주소,사업장명,업태구분명
0,20130201,1,NaN,"광주광역시 서구 치평동 1177-2번지 (106,107호)",뚜레쥬르상무역점,제과점영업
1,20130318,1,NaN,광주광역시 서구 유촌동 868-1번지 (102호),뚜레쥬르광주버들마을점,제과점영업
2,20130417,1,NaN,광주광역시 서구 화정동 1611번지 101호102호103호 상가2동,파리바게뜨화정현대점,제과점영업


In [10]:
# 파생정보 추가 str Accesoor 활용
# str.split() 공백이나 탭 등으로 문자열 분리 -> 서울특별시 종로구 ->[서울특별시,종로구,...]
bread['소재지전체주소'].str.split().str[0] #-> 공백사이에 , 생김 즉 구분 됨 0,1,3,4 0번째 str, 1번째 str ...

0        광주광역시
1        광주광역시
2        광주광역시
3        광주광역시
4        광주광역시
         ...  
52675    울산광역시
52676    광주광역시
52677    부산광역시
52678      경기도
52679      경기도
Name: 소재지전체주소, Length: 52680, dtype: object

In [11]:
bread['업태구분명'].value_counts(dropna=False)

제과점영업    52653
푸드트럭        16
NaN          6
룸살롱          1
과자점          1
한식           1
기타           1
분식           1
Name: 업태구분명, dtype: int64

In [12]:
bread['소재지전체주소'].str.split()

0                   [광주광역시, 서구, 치평동, 1177-2번지, (106,107호)]
1                        [광주광역시, 서구, 유촌동, 868-1번지, (102호)]
2             [광주광역시, 서구, 화정동, 1611번지, 101호102호103호, 상가2동]
3                                [광주광역시, 북구, 운암동, 110-5번지]
4                            [광주광역시, 북구, 오치동, 578-4번지, 1층]
                               ...                        
52675    [울산광역시, 울주군, 삼동면, 보은리, 산, 193-1, 외11필지(1층), 휴게소동]
52676                               [광주광역시, 남구, 월산동, 33-3]
52677                   [부산광역시, 중구, 중앙동7가, 20-1, 롯데백화점광복점]
52678                        [경기도, 군포시, 산본동, 90-7, 1층, 일부]
52679                  [경기도, 성남시, 분당구, 수내동, 114-1, 1층, 일부]
Name: 소재지전체주소, Length: 52680, dtype: object

In [13]:
# 서울소재 제과점영업만 추출
condition1 = bread['소재지전체주소'].str.split().str[0] == '서울특별시'
condition2 = bread['업태구분명'] == '제과점영업'
bread = bread[condition1 & condition2]
bread.head(2)

,인허가일자,상세영업상태코드,폐업일자,소재지전체주소,사업장명,업태구분명
1055,20010713,1,NaN,"서울특별시 은평구 대조동 179-8 1,2층",파리바게뜨(역촌역점),제과점영업
1056,20030507,1,NaN,서울특별시 은평구 갈현동 508-4번지 1층,파리바게뜨,제과점영업


In [14]:
# 원 데이터에서 얼마나 줄었는지 확인
print(df.shape, bread.shape) 

(52680, 48) (12052, 6)


In [15]:
# bread데이터에서 구명, 동명 추출해서 새로운 컬럼으로 추가
# 서울특별시, 은평구, 대조동, 179-8, 1,2층 , 확인해서 0,1,2,
bread['구명'] = bread['소재지전체주소'].str.split().str[1]
bread['동명'] = bread['소재지전체주소'].str.split().str[2]
bread.columns

Index(['인허가일자', '상세영업상태코드', '폐업일자', '소재지전체주소', '사업장명', '업태구분명', '구명', '동명'], dtype='object')

In [16]:
bread.to_csv('./data/bread_t1.csv',index=False)

In [17]:
bread = pd.read_csv('bread_t1.csv')
bread.head(3)

,인허가일자,상세영업상태코드,폐업일자,소재지전체주소,사업장명,업태구분명,구명,동명
0,20010713,1,NaN,"서울특별시 은평구 대조동 179-8 1,2층",파리바게뜨(역촌역점),제과점영업,은평구,대조동
1,20030507,1,NaN,서울특별시 은평구 갈현동 508-4번지 1층,파리바게뜨,제과점영업,은평구,갈현동
2,20021011,1,NaN,서울특별시 은평구 녹번동 278-1번지 (108호),신라명과(역촌현대점),제과점영업,은평구,녹번동


In [18]:
s = pd.Series([pd.NA,np.nan])
s.value_counts(dropna=False)

<NA>    1
NaN     1
dtype: int64

In [19]:
# 폐업일자는 왜 이렇게 많을까? 영업중인 가게는 폐업일자에 표시할 필요가 없으니깐
bread.isna().sum()

인허가일자          0
상세영업상태코드       0
폐업일자        3955
소재지전체주소        0
사업장명           0
업태구분명          0
구명             0
동명             0
dtype: int64

In [20]:
# 아래 코드를 작성하고 윗 코드를 실행하면 폐업일자 결측치가 없는 것을 확인할 수 있다.
bread['폐업일자'] = bread['폐업일자'].fillna(0.0).astype('int64')
bread.dtypes

인허가일자        int64
상세영업상태코드     int64
폐업일자         int64
소재지전체주소     object
사업장명        object
업태구분명       object
구명          object
동명          object
dtype: object

In [21]:
#bread.isna().sum() # 컬럼 별 결측치
bread.isna().sum().sum() # 전체 결측치

0

In [22]:
bread = bread.drop(columns =['소재지전체주소','업태구분명'])
bread.shape

(12052, 6)

In [23]:
# 상세영업상태코드 길어서 상세코드로 변경
bread = bread.rename(columns={'상세영업상태코드':'상세코드'})
bread.head(2)

,인허가일자,상세코드,폐업일자,사업장명,구명,동명
0,20010713,1,0,파리바게뜨(역촌역점),은평구,대조동
1,20030507,1,0,파리바게뜨,은평구,갈현동


In [24]:
bread.to_csv('./data/bread_after.csv',index=False)
bread = pd.read_csv('./data/bread_after.csv')
bread

,인허가일자,상세코드,폐업일자,사업장명,구명,동명
0,20010713,1,0,파리바게뜨(역촌역점),은평구,대조동
1,20030507,1,0,파리바게뜨,은평구,갈현동
2,20021011,1,0,신라명과(역촌현대점),은평구,녹번동
3,20011015,1,0,브르망,은평구,불광동
4,20050406,1,0,케잌하우스밀레,중랑구,중화동
...,...,...,...,...,...,...
12047,20201218,2,20201225,나폴레옹과자점,서초구,반포동
12048,20201218,2,20201225,르몽블랑,서초구,반포동
12049,20210119,2,20210220,리암스(Liam's),서초구,반포동
12050,20210125,2,20210215,유한회사 카페노티드(한시적),강남구,압구정동


#### 3. 서울 어디에 제과점을 오픈할까?

In [25]:
# 설립년도와 폐업년도 확인하기 
# 연산자 계산 나중에 datetime으로 해보기
bread[['인허가일자','폐업일자']]
year = bread['인허가일자']//10000
month = bread['인허가일자']//100 % 100
day = bread['인허가일자'] % 100
bread['설립년도'] =  bread['인허가일자']//10000
bread['폐업년도'] =  bread['폐업일자']//10000
bread.head()

,인허가일자,상세코드,폐업일자,사업장명,구명,동명,설립년도,폐업년도
0,20010713,1,0,파리바게뜨(역촌역점),은평구,대조동,2001,0
1,20030507,1,0,파리바게뜨,은평구,갈현동,2003,0
2,20021011,1,0,신라명과(역촌현대점),은평구,녹번동,2002,0
3,20011015,1,0,브르망,은평구,불광동,2001,0
4,20050406,1,0,케잌하우스밀레,중랑구,중화동,2005,0


In [26]:
s = pd.Series(['20210101','20210330'])
s.str[:4].astype('int64')

0    2021
1    2021
dtype: int64

In [27]:
#datetime 사용
temp = pd.to_datetime(bread['인허가일자'],format='%Y%m%d')
temp.dt.year

0        2001
1        2003
2        2002
3        2001
4        2005
         ... 
12047    2020
12048    2020
12049    2021
12050    2021
12051    2020
Name: 인허가일자, Length: 12052, dtype: int64

In [28]:
# 컬럼 이름 바꿀 때 오타 발생으로 수정
bread = bread.rename(columns={'상세코드':'상태코드'})

In [29]:
# 영업기간 컬럼 추가하기
# 영업기간 = 현재년도 - 설립년도 + 1
# 폐업인 경우에는 폐업년도 - 설립년도 + 1
# nyear = today.year
nyear = 2021
bread.loc[bread['상태코드']==1,'영업기간'] = nyear - bread['설립년도'] + 1
bread.loc[bread['상태코드']==2,'영업기간'] = bread['폐업년도'] - bread['설립년도'] + 1
bread.tail(3)

,인허가일자,상태코드,폐업일자,사업장명,구명,동명,설립년도,폐업년도,영업기간
12049,20210119,2,20210220,리암스(Liam's),서초구,반포동,2021,2021,1.0
12050,20210125,2,20210215,유한회사 카페노티드(한시적),강남구,압구정동,2021,2021,1.0
12051,20201208,2,20210315,고오급 빵집,동작구,사당동,2020,2021,2.0


In [30]:
# 설립년도 별 데이터 수를 구해서 정렬
temp1 = bread['설립년도'].value_counts().sort_index().to_frame().T
temp1

,1952,1965,1966,1967,1968,1969,1970,1971,1972,1973,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
설립년도,1,1,1,11,5,10,20,12,9,5,...,378,431,474,572,644,623,538,546,502,433


In [31]:
temp2 = bread['폐업년도'].value_counts().sort_index().to_frame().T
temp2

,0,1984,1988,1989,1990,1991,1992,1993,1994,1995,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
폐업년도,3955,1,2,1,33,4,35,29,16,24,...,475,374,396,447,469,612,518,514,515,349


In [32]:
temp = pd.concat([temp1,temp2],axis=0,join='outer')
temp

,1952,1965,1966,1967,1968,1969,1970,1971,1972,1973,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,0
설립년도,1.0,1.0,1.0,11.0,5.0,10.0,20.0,12.0,9.0,5.0,...,431,474,572,644,623,538,546,502,433,NaN
폐업년도,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,374,396,447,469,612,518,514,515,349,3955.0


In [33]:
print(bread['사업장명'].to_list())

['파리바게뜨(역촌역점)', '파리바게뜨', '신라명과(역촌현대점)', '브르망', '케잌하우스밀레', '뚜레쥬르', '파리바게뜨(우림시장점)', '뚜레쥬르사가정역점', '몽블랑제(신내점)', '파리바게뜨', '파리바게뜨 용마산역점', '파리바게뜨 중화역점', '델리카리', '파리바게뜨 사가정점', '뚜레쥬르용마산역점', '빵나라', '케익하우스 쥬떼므', '던킨도너츠 먹골점', '파리바게뜨 면목역점', '빵이가득한집', '빵이가득한집', '파리바게뜨면목중앙점', '몽블랑제 면목점', '우림베이커리', '따끈따끈방이야기', '파리바게뜨종암점', '파리바게트', '상제리아베이커리', '투썸플레이스엔타워점', '외계인방앗간 문정점', '앨리스 베이커리 카페', '스위트 팔레트 케이크(sweet palette cake)', '그리리', '도담도담식빵 고덕점', '태극당 District C점', '유한회사 마미갸또금천', '리블리롱제과', '빚은 오금역점', '정수제빵소', '어썸 도넛 서울', '포레포레', '판이판 베이커리', '나폴레옹과자점', '홍미당', '브레드82(Bread82)', '밀포레이트', '브레댄코 신도림역점', '연재스토리즈', '서라당', '미미가미미(米米加味美)', '파리바게뜨(당산포스빌)', '몽쥬 빠티세리(Monge patisserie)', '뚜레쥬르 카페강남대로점', '모찌모찌브레드', '파리크라상 서울대병원점', '수고한 나에게', '르솔레이', '파이앤브라우니', '(주)신세계푸드 트레이더스월계점', '롤링핀 신대방삼거리역점', '프롬프랑(fromfranc)', '그럼,', '좋은 아침 pastry', '서울연인', '(주)파리크라상 파리바게뜨 은평성모병원점', '온베이커리', '153도씨', '브리오슈도레 종로타워점', '브리오슈도레 그랑서울점', '제이몬티', '뚜레쥬르 카페신촌점', '다음에또봐', '기프트(Gift)', '서울연인단팥빵', '슈가파티', '던킨 서울대병원점', '해피데이', '밀브레드(Mill Bread)',

In [34]:
# 파리바게뜨 파리바게트 2개로 표기되어 있음
paris = bread[bread['사업장명'].str.contains('파리바게')]
paris.head(3)

,인허가일자,상태코드,폐업일자,사업장명,구명,동명,설립년도,폐업년도,영업기간
0,20010713,1,0,파리바게뜨(역촌역점),은평구,대조동,2001,0,21.0
1,20030507,1,0,파리바게뜨,은평구,갈현동,2003,0,19.0
6,20060417,1,0,파리바게뜨(우림시장점),중랑구,망우동,2006,0,16.0


In [35]:
tous = bread[bread['사업장명'].str.contains('뚜레쥬르')]
tous.head(3)

,인허가일자,상태코드,폐업일자,사업장명,구명,동명,설립년도,폐업년도,영업기간
5,20050412,1,0,뚜레쥬르,중랑구,신내동,2005,0,17.0
7,20060822,1,0,뚜레쥬르사가정역점,중랑구,면목동,2006,0,16.0
14,20070416,1,0,뚜레쥬르용마산역점,중랑구,면목동,2007,0,15.0


In [36]:
print(paris.shape,tous.shape)

(1059, 9) (586, 9)


In [37]:
temp1 = paris.loc[(paris['설립년도']>=2000)&(paris['상태코드']==1),'영업기간']
temp2 = tous.loc[(tous['설립년도']>=2000)&(tous['상태코드']==1),'영업기간']
# 영업기간 평균 구하기
s = pd.Series([temp1.mean(), temp2.mean()],index=['파리바게트','뚜레쥬르'])
comp = pd.DataFrame(s,columns=['영업'])
comp

,영업
파리바게트,12.357022
뚜레쥬르,9.796296


In [38]:
temp1 = paris.loc[(paris['설립년도']>=2000)&(paris['상태코드']==2),'영업기간']
temp2 = tous.loc[(tous['설립년도']>=2000)&(tous['상태코드']==2),'영업기간']

comp['폐업'] = [temp1.mean(),temp2.mean()]
comp

,영업,폐업
파리바게트,12.357022,7.712687
뚜레쥬르,9.796296,7.309028


In [39]:
# 파리바게트, 뚜레쥬르 이외 제과점(other)들 영업기간은
condition1 = (bread['사업장명'].str.contains('파리바게'))
condition2 = (bread['사업장명'].str.contains('뚜레쥬르'))
other = bread.loc[~(condition1 | condition2), :]# ~ 아닌 것은
other.shape

(10407, 9)

In [40]:
print('전체:',bread.shape[0])
print('파바:',paris.shape[0])
print('뚜쥬:',tous.shape[0])
print('나머지:',other.shape[0])

전체: 12052
파바: 1059
뚜쥬: 586
나머지: 10407


In [41]:
temp1 = other.loc[(other['설립년도']>=2000)&(other['상태코드']==1),'영업기간']
temp2 = other.loc[(other['설립년도']>=2000)&(other['상태코드']==2),'영업기간']

temp = pd.DataFrame([[temp1.mean(), temp2.mean()]], index=['나머지'], columns=['영업', '폐업'])
comp = comp.append(temp)
comp

C:\Users\choic\AppData\Local\Temp\ipykernel_5960\1153167289.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  comp = comp.append(temp)


,영업,폐업
파리바게트,12.357022,7.712687
뚜레쥬르,9.796296,7.309028
나머지,6.332259,4.619193


In [48]:
# 제과점 별 위치 분석 -> 구 별 매장 개수
temp = paris.loc[paris['상태코드']==1, '구명'].value_counts().to_frame()
temp.head(2)

,구명
강남구,59
송파구,49


In [49]:
temp['폐업'] = paris.loc[paris['상태코드']==2, '구명'].value_counts()
temp2 = temp.reset_index()
temp2.head(2)

,index,구명,폐업
0,강남구,59,28
1,송파구,49,20


In [50]:
temp2.columns = ['구','영업','폐업']

In [51]:
# 폐업비율표 만들기 
# 폐업비율 = 폐업/(폐업+영업) * 100
temp2['폐업비율'] = (temp2['폐업'] / (temp2['폐업']+temp2['영업'])) *100
temp2.head()

,구,영업,폐업,폐업비율
0,강남구,59,28,32.183908
1,송파구,49,20,28.985507
2,서초구,38,24,38.709677
3,영등포구,37,22,37.288136
4,강서구,34,13,27.659574


In [54]:
temp2.sort_values('폐업비율').head()

,구,영업,폐업,폐업비율
6,은평구,33,6,15.384615
12,동대문구,27,5,15.625000
17,도봉구,21,4,16.000000
8,구로구,30,10,25.000000
23,강북구,16,6,27.272727


In [55]:
s = pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
s.quantile([0, 0.5, 1])

0.0     1.0
0.5     5.5
1.0    10.0
dtype: float64

In [56]:
value = temp2['폐업비율'].quantile(0.3)
print(value)
temp2.loc[temp2['폐업비율'] <= value, ].sort_values('폐업비율')

29.042064333686817


,구,영업,폐업,폐업비율
6,은평구,33,6,15.384615
12,동대문구,27,5,15.625000
17,도봉구,21,4,16.000000
8,구로구,30,10,25.000000
23,강북구,16,6,27.272727
4,강서구,34,13,27.659574
15,성동구,26,10,27.777778
1,송파구,49,20,28.985507


In [57]:
value = temp2['폐업비율'].quantile(0.8)
print(value)
temp2.loc[temp2['폐업비율'] >= value, ]

42.37726098191215


,구,영업,폐업,폐업비율
11,강동구,28,25,47.169811
18,용산구,20,16,44.444444
20,광진구,19,16,45.714286
22,금천구,17,14,45.161290
24,종로구,12,12,50.000000
